In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from requests.exceptions import Timeout
import re
import time

In [5]:
leagues = {
    'Premier League': "https://en.wikipedia.org/wiki/2024%E2%80%9325_Premier_League",
    'Bundesliga': "https://en.wikipedia.org/wiki/2024%E2%80%9325_Bundesliga",
    'La Liga': "https://en.wikipedia.org/wiki/2024%E2%80%9325_La_Liga",
    'Serie A': "https://en.wikipedia.org/wiki/2024%E2%80%9325_Serie_A",
    'Ligue 1': "https://en.wikipedia.org/wiki/2024%E2%80%9325_Ligue_1"
}

In [7]:
def get_with_retry(url, retries=3, delay=5, timeout=10):
    for i in range(retries):
        try:
            response = requests.get(url, timeout=timeout)
            response.raise_for_status()  # Raise an HTTPError for bad responses
            return response
        except Timeout:
            print(f"Timeout occurred, retrying... ({i+1}/{retries})")
            time.sleep(delay)
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            break
    return None

def dms_to_decimal(dms_str):
    if not isinstance(dms_str, str):
        return None  # Return None if the input is not a string
    
    # Define regex to extract degrees, minutes, seconds, and direction
    dms_regex = re.compile(r'(\d+\.?\d*)°\s*(\d+\'?)?\s*(\d+\.?\d*)?"?\s*([NSEW])')
    
    matches = dms_regex.findall(dms_str)
    if not matches:
        return None
    
    decimal_degrees = 0.0
    
    for match in matches:
        degrees, minutes, seconds, direction = match
        degrees = float(degrees)
        minutes = float(minutes.replace("'", "")) if minutes else 0.0
        seconds = float(seconds) if seconds else 0.0
        
        decimal = degrees + (minutes / 60) + (seconds / 3600)
        
        # If direction is S or W, make it negative
        if direction in ['S', 'W']:
            decimal = -decimal
        
        decimal_degrees = decimal
    
    return decimal_degrees

def get_coordinates_from_city_page(city_link):
    """Helper function to extract and convert coordinates from a city page."""
    city_response = requests.get(city_link)
    city_soup = BeautifulSoup(city_response.content, 'html.parser')
    
    # Find the coordinates on the city page
    coordinates_tag = city_soup.find('span', {'class': 'geo-dec'}) or city_soup.find('span', {'class': 'geo'})
    if coordinates_tag:
        coordinates = coordinates_tag.get_text(strip=True)
        
        # Check if the coordinates are in DMS format
        if '°' in coordinates or '′' in coordinates or '″' in coordinates:
            lat_long = coordinates.split(" ")
            lat = dms_to_decimal(lat_long[0])
            long = dms_to_decimal(lat_long[1])
        else:
            lat, long = coordinates.split(",")
        
        return lat, long
    
    return None, None

all_players_data = []

for league_name, league_url in leagues.items():
    response = requests.get(league_url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Select the correct table: 
    teams_table = soup.find_all('table', {'class': 'wikitable'})[0 if league_name == 'Premier League' else 1]
    
    teams = []

    for row in teams_table.find_all('tr')[1:]:
        cells = row.find_all('td')
        if len(cells) > 0:
            team_name = cells[0].get_text(strip=True)
            print(team_name)
            team_link = "https://en.wikipedia.org" + cells[0].find('a')['href']
            teams.append({'team_name': team_name, 'team_link': team_link})

    # Scrape each team's player data
    for team in teams:
        response = requests.get(team['team_link'])
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract team logo
        logo_tag = soup.find('td', class_='infobox-image')
        if logo_tag:
            img_tag = logo_tag.find('img')
            if img_tag:
                logo_url = "https:" + img_tag['src']
                team['logo_url'] = logo_url
                print(f"Team: {team['team_name']}, Logo URL: {team['logo_url']}")
            else:
                team['logo_url'] = None
        else:
            team['logo_url'] = None

        # Process the rest of the team data...
        squad_tables = soup.find_all('table', {'class': 'wikitable football-squad nogrid'})

        for table_index, table in enumerate(squad_tables):
            # Determine the label based on the table index
            if table_index == 0:
                squad_label = "First Team"
            else:
                squad_label = "On Loan"

            for row in table.find_all('tr')[1:]:
                player_cell = row.find('span', {'class': 'fn'})  # Find the player cell
                player_link_tag = player_cell.find('a') if player_cell else None
                if player_link_tag:
                    player_name = player_link_tag.get_text(strip=True)
                    player_link = "https://en.wikipedia.org" + player_link_tag['href']
                    players_data = {
                        'league': league_name,
                        'team': team['team_name'],
                        'team_logo': team['logo_url'],
                        'player_name': player_name,
                        'player_link': player_link,
                        'squad_status': squad_label,
                        'team_logo_url': team['logo_url']
                    }
                    all_players_data.append(players_data)
                    
totalplayers = len(all_players_data)
playerscompleted = 1

for player in all_players_data:
    response = get_with_retry(player['player_link'])
    
    if response is not None:
        soup = BeautifulSoup(response.content, 'html.parser')

        infobox = soup.find('table', {'class': 'infobox'})

        if infobox:
            rows = infobox.find_all('tr')

            for row in rows:
                header = row.find('th')
                if header:
                    header_text = header.get_text(strip=True)
                    if header_text == 'Place of birth':
                        player['place_of_birth'] = row.find('td').get_text(strip=True)
                        
                        # Extract the primary city link
                        city_link_tag = row.find('td').find('a', href=True)
                        
                        if city_link_tag:  # Check if city_link_tag is not None
                            city_link = "https://en.wikipedia.org" + city_link_tag['href']
                            
                            # Check if the primary link is a redlink (non-existent page)
                            if 'new' in city_link_tag.get('class', []):
                                # Look for a secondary link in the brackets
                                td_tag = row.find('td')
                                if td_tag:
                                    span_tag = td_tag.find('span', class_='noprint')
                                    if span_tag:
                                        secondary_link_tag = span_tag.find('a', class_='extiw')
                                        if secondary_link_tag:
                                            city_link = secondary_link_tag['href']
                            
                            # Navigate to the city page (either primary or secondary link) to find coordinates
                            lat, long = get_coordinates_from_city_page(city_link)
                            player['latitude'] = lat
                            player['longitude'] = long
                    elif header_text == 'Full name':
                        player['full_name'] = row.find('td').get_text(strip=True)
                    elif header_text == 'Date of birth':
                        player['date_of_birth'] = row.find('td').get_text(strip=True)
                    elif header_text == 'Height':
                        player['height'] = row.find('td').get_text(strip=True)
                    elif header_text == 'Position(s)':
                        player['position_full'] = row.find('td').get_text(strip=True)
                        
        print(f"{playerscompleted}/{totalplayers} Completed. {player['player_name']} playing at {player['team']}")
        playerscompleted += 1
        
    else:
        print(f"Failed to retrieve the page for {player['player_name']} at {player['player_link']}")
        
        playerscompleted += 1
              
# Convert the player data to a pandas DataFrame
df = pd.DataFrame(all_players_data)

# Save the DataFrame to a CSV file
df.to_csv('big_five_leagues_players.csv', index=False)

Arsenal
Aston Villa
Bournemouth
Brentford
Brighton & Hove Albion
Chelsea
Crystal Palace
Everton
Fulham
Ipswich Town
Leicester City
Liverpool
Manchester City
Manchester United
Newcastle United
Nottingham Forest
Southampton
Tottenham Hotspur
West Ham United
Wolverhampton Wanderers
Team: Arsenal, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/5/53/Arsenal_FC.svg/180px-Arsenal_FC.svg.png
Team: Aston Villa, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/9/9a/Aston_Villa_FC_new_crest.svg/150px-Aston_Villa_FC_new_crest.svg.png
Team: Bournemouth, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/e/e5/AFC_Bournemouth_%282013%29.svg/150px-AFC_Bournemouth_%282013%29.svg.png
Team: Brentford, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/2/2a/Brentford_FC_crest.svg/180px-Brentford_FC_crest.svg.png
Team: Brighton & Hove Albion, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/d/d0/Brighton_and_Hove_Albion_FC_crest.svg/190px-Brighton_and_Hove_Albion_

Team: Real Betis, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/1/13/Real_betis_logo.svg/220px-Real_betis_logo.svg.png
Team: Real Madrid, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/5/56/Real_Madrid_CF.svg/160px-Real_Madrid_CF.svg.png
Team: Real Sociedad, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/f/f1/Real_Sociedad_logo.svg/175px-Real_Sociedad_logo.svg.png
Team: Sevilla, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/3/3b/Sevilla_FC_logo.svg/155px-Sevilla_FC_logo.svg.png
Team: Valencia, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/c/ce/Valenciacf.svg/176px-Valenciacf.svg.png
Team: Valladolid, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/a/a9/Real_Valladolid_CF_crest.svg/195px-Real_Valladolid_CF_crest.svg.png
Team: Villarreal, Logo URL: https://upload.wikimedia.org/wikipedia/en/thumb/b/b9/Villarreal_CF_logo-en.svg/170px-Villarreal_CF_logo-en.svg.png
Atalanta
Bologna
Cagliari
Como
Empoli
Fiorentina
Genoa
Hella

31/3374 Completed. Matty Cash playing at Aston Villa
32/3374 Completed. Diego Carlos playing at Aston Villa
33/3374 Completed. Ezri Konsa playing at Aston Villa
34/3374 Completed. Tyrone Mings playing at Aston Villa
35/3374 Completed. Ross Barkley playing at Aston Villa
36/3374 Completed. John McGinn playing at Aston Villa
37/3374 Completed. Youri Tielemans playing at Aston Villa
38/3374 Completed. Jhon Durán playing at Aston Villa
39/3374 Completed. Emiliano Buendía playing at Aston Villa
40/3374 Completed. Ollie Watkins playing at Aston Villa
41/3374 Completed. Lucas Digne playing at Aston Villa
42/3374 Completed. Pau Torres playing at Aston Villa
43/3374 Completed. Joe Gauci playing at Aston Villa
44/3374 Completed. Jaden Philogene playing at Aston Villa
45/3374 Completed. Kosta Nedeljković playing at Aston Villa
46/3374 Completed. Ian Maatsen playing at Aston Villa
47/3374 Completed. Emiliano Martínez playing at Aston Villa
48/3374 Completed. Amadou Onana playing at Aston Villa
49/

173/3374 Completed. Cameron Peupion playing at Brighton & Hove Albion
174/3374 Completed. Joël Veltman playing at Brighton & Hove Albion
175/3374 Completed. Jack Hinshelwood playing at Brighton & Hove Albion
176/3374 Completed. Ferdi Kadıoğlu playing at Brighton & Hove Albion
177/3374 Completed. Jeremy Sarmiento playing at Brighton & Hove Albion
178/3374 Completed. Valentin Barco playing at Brighton & Hove Albion
179/3374 Completed. Abdallah Sima playing at Brighton & Hove Albion
180/3374 Completed. Amario Cozier-Duberry playing at Brighton & Hove Albion
181/3374 Completed. Andrew Moran playing at Brighton & Hove Albion
182/3374 Completed. Carl Rushworth playing at Brighton & Hove Albion
183/3374 Completed. Malick Yalcouyé playing at Brighton & Hove Albion
184/3374 Completed. Tom McGill playing at Brighton & Hove Albion
185/3374 Completed. Kjell Scherpen playing at Brighton & Hove Albion
186/3374 Completed. Facundo Buonanotte playing at Brighton & Hove Albion
187/3374 Completed. Ibrahi

317/3374 Completed. Jorge Cuenca playing at Fulham
318/3374 Completed. Sander Berge playing at Fulham
319/3374 Completed. Alex Iwobi playing at Fulham
320/3374 Completed. Andreas Pereira playing at Fulham
321/3374 Completed. Saša Lukić playing at Fulham
322/3374 Completed. Timothy Castagne playing at Fulham
323/3374 Completed. Steven Benda playing at Fulham
324/3374 Completed. Ryan Sessegnon playing at Fulham
325/3374 Completed. Issa Diop playing at Fulham
326/3374 Completed. Emile Smith Rowe playing at Fulham
327/3374 Completed. Antonee Robinson playing at Fulham
328/3374 Completed. Reiss Nelson playing at Fulham
329/3374 Completed. Carlos Vinícius playing at Fulham
330/3374 Completed. Luke Harris playing at Fulham
331/3374 Completed. Arijanet Muric playing at Ipswich Town
332/3374 Completed. Harry Clarke playing at Ipswich Town
333/3374 Completed. Leif Davis playing at Ipswich Town
334/3374 Completed. Sam Morsy playing at Ipswich Town
335/3374 Completed. Luke Woolfenden playing at Ip

...

2827/3374 Completed. Ki-Jana Hoever playing at Auxerre
2828/3374 Completed. Ange-Loïc N'Gatta playing at Auxerre
2829/3374 Completed. Hamed Traorè playing at Auxerre
2830/3374 Completed. Paul Joly playing at Auxerre
2831/3374 Completed. Kévin Danois playing at Auxerre
2832/3374 Completed. Raphaël Adicéam playing at Auxerre
2833/3374 Completed. Théo De Percin playing at Auxerre
2834/3374 Completed. Elisha Owusu playing at Auxerre
2835/3374 Completed. Ado Onaiwu playing at Auxerre
2836/3374 Completed. Rayan Raveloson playing at Auxerre
2837/3374 Completed. Ousmane Camara playing at Auxerre
2838/3374 Completed. Bradley Locko playing at Brest
2839/3374 Completed. Abdoulaye Ndiaye playing at Brest
2840/3374 Completed. Brendan Chardonnet playing at Brest
2841/3374 Completed. Hugo Magnetti playing at Brest
2842/3374 Completed. Romain Del Castillo playing at Brest
2843/3374 Completed. Axel Camblan playing at Brest
2844/3374 Completed. Luck Zogbé playing at Brest
2845/3374 Completed. Abdallah S

2975/3374 Completed. Saël Kumbedi playing at Lyon
2976/3374 Completed. Clinton Mata playing at Lyon
2977/3374 Completed. Lucas Perri playing at Lyon
2978/3374 Completed. Warmed Omari playing at Lyon
2979/3374 Completed. Florent Sanchez playing at Lyon
Request failed: 404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Justin_Bengui
Failed to retrieve the page for Justin Bengui at https://en.wikipedia.org/w/index.php?title=Justin_Bengui&action=edit&redlink=1
2981/3374 Completed. Nemanja Matić playing at Lyon
2982/3374 Completed. Mahamadou Diawara playing at Lyon
2983/3374 Completed. Ernest Nuamah playing at Lyon
2984/3374 Completed. Rémy Descamps playing at Lyon
2985/3374 Completed. Duje Ćaleta-Car playing at Lyon
2986/3374 Completed. Georges Mikautadze playing at Lyon
2987/3374 Completed. Ainsley Maitland-Niles playing at Lyon
2988/3374 Completed. Mathieu Patouillet playing at Lyon
2989/3374 Completed. Achraf Laâziri playing at Lyon
Request failed: 404 Client Error: Not 

3051/3374 Completed. Michael Murillo playing at Marseille
3052/3374 Completed. Chancel Mbemba playing at Marseille
3053/3374 Completed. Roggerio Nyakossi playing at Marseille
3054/3374 Completed. Stéphane Sparagna playing at Marseille
3055/3374 Completed. Raimane Daou playing at Marseille
3056/3374 Completed. Sofiane Sidi Ali playing at Marseille
3057/3374 Completed. Pau López playing at Marseille
3058/3374 Completed. Simon Ngapandouetnbu playing at Marseille
3059/3374 Completed. Radosław Majecki playing at Monaco
3060/3374 Completed. Vanderson playing at Monaco
3061/3374 Completed. Jordan Teze playing at Monaco
3062/3374 Completed. Thilo Kehrer playing at Monaco
3063/3374 Completed. Denis Zakaria playing at Monaco
3064/3374 Completed. Eliesse Ben Seghir playing at Monaco
3065/3374 Completed. Eliot Matazo playing at Monaco
3066/3374 Completed. Folarin Balogun playing at Monaco
3067/3374 Completed. Aleksandr Golovin playing at Monaco
3068/3374 Completed. Maghnes Akliouche playing at Mon

3197/3374 Completed. Milan Škriniar playing at Paris Saint-Germain
3198/3374 Completed. Matvey Safonov playing at Paris Saint-Germain
3199/3374 Completed. Yoram Zague playing at Paris Saint-Germain
3200/3374 Completed. Willian Pacho playing at Paris Saint-Germain
3201/3374 Completed. Arnau Tenas playing at Paris Saint-Germain
3202/3374 Completed. João Neves playing at Paris Saint-Germain
3203/3374 Completed. Lucas Lavallée playing at Paris Saint-Germain
3204/3374 Completed. Juan Bernat playing at Paris Saint-Germain
3205/3374 Completed. Nordi Mukiele playing at Paris Saint-Germain
3206/3374 Completed. Gabriel Moscardo playing at Paris Saint-Germain
3207/3374 Completed. Cher Ndour playing at Paris Saint-Germain
3208/3374 Completed. Renato Sanches playing at Paris Saint-Germain
3209/3374 Completed. Xavi Simons playing at Paris Saint-Germain
3210/3374 Completed. Carlos Soler playing at Paris Saint-Germain
3211/3374 Completed. Ilyes Housni playing at Paris Saint-Germain
3212/3374 Completed

3330/3374 Completed. Anthony Briançon playing at Saint-Étienne
3331/3374 Completed. Ibrahima Wadji playing at Saint-Étienne
3332/3374 Completed. Lamine Fomba playing at Saint-Étienne
3333/3374 Completed. Yvann Maçon playing at Saint-Étienne
3334/3374 Completed. Igor Miladinović playing at Saint-Étienne
3335/3374 Completed. Aïmen Moueffek playing at Saint-Étienne
3336/3374 Completed. Gautier Larsonneur playing at Saint-Étienne
3337/3374 Completed. Lucas Stassin playing at Saint-Étienne
3338/3374 Completed. Mahmoud Bentayg playing at Saint-Étienne
Request failed: 404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Darling_Bladi
Failed to retrieve the page for Darling Bladi at https://en.wikipedia.org/w/index.php?title=Darling_Bladi&action=edit&redlink=1
3340/3374 Completed. Karim Cissé playing at Saint-Étienne
3341/3374 Completed. Antoine Gauthier playing at Saint-Étienne
Request failed: 404 Client Error: Not Found for url: https://en.wikipedia.org/wiki/Marwann_N%27Zuzi
Fa